In [1]:
#FAISS Facebook AI Similarity Search DB

In [1]:
!pip -q install langchain_ollama langchain_community faiss-cpu langchain-openai langchain-text-splitters
!pip install -q langchain_huggingface

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepspeed 0.10.3 requires pydantic<2.0.0, but you have pydantic 2.10.6 which is incompatible.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
gradient 2.0.6 requires marshmallow<3.0, but you have marshmallow 3.26.1 which is incompatible.


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter


In [3]:
loader=TextLoader('speech.txt')
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=200,
                                    chunk_overlap=30)
docs=text_splitter.split_documents(documents)

Created a chunk of size 599, which is longer than the specified 200
Created a chunk of size 617, which is longer than the specified 200
Created a chunk of size 744, which is longer than the specified 200
Created a chunk of size 299, which is longer than the specified 200
Created a chunk of size 452, which is longer than the specified 200
Created a chunk of size 777, which is longer than the specified 200
Created a chunk of size 260, which is longer than the specified 200
Created a chunk of size 530, which is longer than the specified 200
Created a chunk of size 888, which is longer than the specified 200


In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")


In [5]:
HF_embed=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db=FAISS.from_documents(docs, HF_embed)

2025-02-04 22:01:09.742367: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-04 22:01:09.742465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-04 22:01:09.744401: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 22:01:09.754644: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-04 22:01:10.959245: W tensorflow/compiler/tf2

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
##query
query="What does the speaker believe is the main reason for US to enter the war?"

In [7]:
docs=db.similarity_search(query)
print(docs)

[Document(id='35fdc516-9007-405b-947b-3fcca8d75c7e', metadata={'source': 'speech.txt'}, page_content="Speech is the use of the human voice as a medium for language. Spoken language combines vowel and consonant sounds to form units of meaning like words, which belong to a language's lexicon. There are many different intentional speech acts, such as informing, declaring, asking, persuading, directing; acts may vary in various aspects like enunciation, intonation, loudness, and tempo to convey meaning. Individuals may also unintentionally communicate aspects of their social position through speech, such as sex, age, place of origin, physiological and mental condition, education, and experiences."), Document(id='bdef3213-e204-41bd-9d94-0c62a8442e86', metadata={'source': 'speech.txt'}, page_content='Evolution\nMain article: Origin of speech\nAlthough related to the more general problem of the origin of language, the evolution of distinctively human speech capacities has become a distinct an

In [8]:
db.save_local("faiss_index")

In [11]:
new_db=FAISS.load_local("faiss_index",HF_embed, allow_dangerous_deserialization=True)

In [12]:
docs=new_db.similarity_search(query)

In [13]:
docs

[Document(id='35fdc516-9007-405b-947b-3fcca8d75c7e', metadata={'source': 'speech.txt'}, page_content="Speech is the use of the human voice as a medium for language. Spoken language combines vowel and consonant sounds to form units of meaning like words, which belong to a language's lexicon. There are many different intentional speech acts, such as informing, declaring, asking, persuading, directing; acts may vary in various aspects like enunciation, intonation, loudness, and tempo to convey meaning. Individuals may also unintentionally communicate aspects of their social position through speech, such as sex, age, place of origin, physiological and mental condition, education, and experiences."),
 Document(id='bdef3213-e204-41bd-9d94-0c62a8442e86', metadata={'source': 'speech.txt'}, page_content='Evolution\nMain article: Origin of speech\nAlthough related to the more general problem of the origin of language, the evolution of distinctively human speech capacities has become a distinct a